In [1]:
import pandas as pd
import urllib.request
import re
import os
from datetime import datetime


Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу. Ця процедура має бути автоматизована, параметром процедури має бути індекс (номер) області. При зберіганні файлу до його імені потрібно додати дату та час завантаження;

In [2]:

def get_data(i):
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={i}&year1=1981&year2=2024&type=Mean"
    vhi_url = urllib.request.urlopen(url)
    data = vhi_url.read().decode('utf-8')
    data = re.sub(r'<[^>]*>', '', data)
    now = datetime.now()
    time = now.strftime("%d%m%Y%H%M%S")

    if not os.path.exists('data'):
        os.makedirs('data')

    filename = f'data/obl_{i}_{time}.csv'
    dir = os.listdir('data')
    for filename_old in dir:
        if filename_old.startswith(f'obl_{i}_'):
            os.rename(os.path.join('data', filename_old), filename)
            break
    
    with open(filename, 'w') as file:
        file.write(data)

for i in range(1,28):
    if i in [12,20]:
        continue
    get_data(i)



Зчитати завантажені текстові файли у фрейм (детальніше про роботу із фреймами буде розказано у подальших лабораторних роботах). Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли;

In [3]:
def create_df(path):  
    file_list = os.listdir(path)
    file_list = sorted(file_list, key=lambda filename: int(filename.split("_")[1]))
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'index']
    df = pd.DataFrame(columns=headers)
    index = 1
    for i in file_list:
        temp_df = pd.read_csv(os.path.join(path, i), header=1, names=headers)
        temp_df.fillna({"index":index}, inplace=True)
        df = pd.concat([df, temp_df], ignore_index=True)
        index+=1
    return df

df = create_df("data")
df

C:\Users\dddddd\AppData\Local\Temp\ipykernel_17448\2694371073.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, temp_df], ignore_index=True)


,Year,Week,SMN,SMT,VCI,TCI,VHI,index
0,1982,1,0.053,260.31,45.01,39.46,42.23,1.0
1,1982,2,0.054,262.29,46.83,31.75,39.29,1.0
2,1982,3,0.055,263.82,48.13,27.24,37.68,1.0
3,1982,4,0.053,265.33,46.09,23.91,35.00,1.0
4,1982,5,0.050,265.66,41.46,26.65,34.06,1.0
...,...,...,...,...,...,...,...,...
55895,2024,48,-1.000,-1.00,-1.00,-1.00,-1.00,25.0
55896,2024,49,-1.000,-1.00,-1.00,-1.00,-1.00,25.0
55897,2024,50,-1.000,-1.00,-1.00,-1.00,-1.00,25.0
55898,2024,51,-1.000,-1.00,-1.00,-1.00,-1.00,25.0


Реалізувати процедуру, яка змінить індекси областей, які використані на порталі NOAA на наступні:

In [4]:
indexs = {1:22, 2:24, 3:23, 4:25, 5:3, 6:4, 7:8, 8:19, 9:20, 10:21, 11:9, 12:10, 13:11, 14:12, 15:13, 16:14, 17:15, 18:16, 19:17, 20:18, 21:6, 22:1, 23:2, 24:7, 25:5}
df.replace(indexs, inplace=True)
df

C:\Users\dddddd\AppData\Local\Temp\ipykernel_17448\2679290283.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(indexs, inplace=True)


,Year,Week,SMN,SMT,VCI,TCI,VHI,index
0,1982,22,0.053,260.31,45.01,39.46,42.23,22.0
1,1982,24,0.054,262.29,46.83,31.75,39.29,22.0
2,1982,23,0.055,263.82,48.13,27.24,37.68,22.0
3,1982,25,0.053,265.33,46.09,23.91,35.00,22.0
4,1982,3,0.050,265.66,41.46,26.65,34.06,22.0
...,...,...,...,...,...,...,...,...
55895,2024,48,-1.000,-1.00,-1.00,-1.00,-1.00,5.0
55896,2024,49,-1.000,-1.00,-1.00,-1.00,-1.00,5.0
55897,2024,50,-1.000,-1.00,-1.00,-1.00,-1.00,5.0
55898,2024,51,-1.000,-1.00,-1.00,-1.00,-1.00,5.0


Ряд VHI для області за вказаний рік, пошук екстремумів (min та max);

In [5]:
def find_VHI_extremes_by_year(year, index):
    vhi_values = df.loc[(df["Year"]==year) & (df["index"]==index), 'VHI']
    for VHI in vhi_values:
        print(VHI)
    print("min", vhi_values.min())
    print("max", vhi_values.max())

find_VHI_extremes_by_year(1982, 22)


42.23
39.29
37.68
35.0
34.06
33.01
31.65
31.4
31.73
31.44
30.8
31.89
33.37
34.2
37.78
43.81
49.85
54.97
55.86
54.59
55.21
55.14
53.64
54.09
56.84
58.43
59.17
61.11
63.42
64.58
62.83
60.01
58.51
57.09
55.47
53.11
50.1
49.19
47.97
45.43
41.11
36.35
34.76
28.95
24.18
23.83
25.6
29.7
31.94
33.29
36.61
37.15
min 23.83
max 64.58


Ряд VHI за вказаний діапазон років для вказаних областей; виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей по Україні (20% областей - 5 областей з 25);

In [6]:
def find_VHI_by_years_and_regions(start_year, end_year, regions, p=20):
    for i in regions:
        year =start_year
        while year <= end_year:
            print(f"VHI для регону {i} за рік {year}")
            VHI = df.loc[(df["Year"] == year)&(df["index"]==i), "VHI"]
            year+=1
            print(VHI.tolist())
        print("================================================================================")
    
    df_drought = df[(df["VHI"]<=15)&(df["VHI"]!=-1)]
    years = df_drought["Year"].tolist()
    years = list(set(years))
    for i in years:
        obl_list = df_drought.loc[(df_drought["Year"]==i), "index"].tolist()
        obl_list = list(set(obl_list))
        if (len(obl_list)*100//25)>=p:
            print (f"протягом {i} року eкстимальні посухи торкнулися більше {p}% областей України")



find_VHI_by_years_and_regions(1982, 1985, [22, 3], 20)

VHI для регону 22 за рік 1982
[42.23, 39.29, 37.68, 35.0, 34.06, 33.01, 31.65, 31.4, 31.73, 31.44, 30.8, 31.89, 33.37, 34.2, 37.78, 43.81, 49.85, 54.97, 55.86, 54.59, 55.21, 55.14, 53.64, 54.09, 56.84, 58.43, 59.17, 61.11, 63.42, 64.58, 62.83, 60.01, 58.51, 57.09, 55.47, 53.11, 50.1, 49.19, 47.97, 45.43, 41.11, 36.35, 34.76, 28.95, 24.18, 23.83, 25.6, 29.7, 31.94, 33.29, 36.61, 37.15]
VHI для регону 22 за рік 1983
[35.52, 34.61, 36.36, 38.56, 39.88, 41.0, 41.47, 42.13, 42.32, 41.66, 38.52, 37.59, 36.93, 35.08, 34.32, 35.7, 37.56, 40.77, 44.91, 45.97, 45.58, 45.56, 46.21, 46.41, 45.4, 43.97, 43.87, 43.7, 43.03, 43.33, 45.11, 46.6, 47.56, 47.19, 45.62, 44.78, 43.49, 42.43, 44.1, 45.03, 43.21, 43.56, 44.56, 42.81, 41.05, 39.06, 41.01, 43.91, 43.25, 43.06, 43.44, 44.63]
VHI для регону 22 за рік 1984
[45.35, 45.72, 47.44, 47.25, 45.7, 44.98, 44.79, 47.0, 47.06, 45.26, 41.69, 38.52, 35.07, 31.19, 30.24, 33.0, 35.58, 39.84, 45.94, 50.66, 54.94, 57.09, 59.94, 64.01, 67.15, 68.92, 69.64, 70.64,

Аналогічно для помірних посух

In [7]:
def find_years_of_moderate_droughts(p=20):
    df_drought = df[(df["VHI"]>15)&(df["VHI"]!=-1)&(df["VHI"]<=35)]
    years = df_drought["Year"].tolist()
    years = list(set(years))
    for i in years:
        obl_list = df_drought.loc[(df_drought["Year"]==i), "index"].tolist()
        obl_list = list(set(obl_list))
        if (len(obl_list)*100//25)>=p:
            print (f"протягом {i} року помірні посухи торкнулися більше {p}% областей України")

find_years_of_moderate_droughts(70)

протягом 1982 року помірні посухи торкнулися більше 70% областей України
протягом 1983 року помірні посухи торкнулися більше 70% областей України
протягом 1984 року помірні посухи торкнулися більше 70% областей України
протягом 1985 року помірні посухи торкнулися більше 70% областей України
протягом 1986 року помірні посухи торкнулися більше 70% областей України
протягом 1987 року помірні посухи торкнулися більше 70% областей України
протягом 1988 року помірні посухи торкнулися більше 70% областей України
протягом 1989 року помірні посухи торкнулися більше 70% областей України
протягом 1990 року помірні посухи торкнулися більше 70% областей України
протягом 1991 року помірні посухи торкнулися більше 70% областей України
протягом 1992 року помірні посухи торкнулися більше 70% областей України
протягом 1993 року помірні посухи торкнулися більше 70% областей України
протягом 1994 року помірні посухи торкнулися більше 70% областей України
протягом 1995 року помірні посухи торкнулися більше